The goal of this script is to get all the unique substrates from a file and then try to filter out co-factors and other co-substrates.<br/><br/>Copyright (C) 2017  Martin Engqvist Lab<br/>This program is free software: you can redistribute it and/or modify<br/>it under the terms of the GNU General Public License as published by<br/>the Free Software Foundation, either version 3 of the License, or<br/>(at your option) any later version.<br/>This program is distributed in the hope that it will be useful,<br/>but WITHOUT ANY WARRANTY; without even the implied warranty of<br/>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br/>GNU General Public License for more details.<br/>You should have received a copy of the GNU General Public License<br/>along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [43]:
import os
from dotenv import load_dotenv, find_dotenv
from os.path import join, dirname, basename, exists, isdir

### Load environmental variables from the project root directory ###
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

# now you can get the variables using their names

# Check whether a network drive has been specified
DATABASE = os.environ.get("NETWORK_URL")
if DATABASE == 'None':
    pass
else:
    pass
    #mount network drive here

# set up directory paths
CURRENT_DIR = os.getcwd()
PROJ = dirname(dotenv_path) # project root directory

DATA = join(PROJ, 'data') #data directory
RAW_EXTERNAL = join(DATA, 'raw_external') # external data raw directory
RAW_INTERNAL = join(DATA, 'raw_internal') # internal data raw directory
INTERMEDIATE = join(DATA, 'intermediate') # intermediate data directory
FINAL = join(DATA, 'final') # final data directory

RESULTS = join(PROJ, 'results') # output directory
FIGURES = join(RESULTS, 'figures') # figure output directory
PICTURES = join(RESULTS, 'pictures') # picture output directory


# make folders specific for certain data
folder_name = 'brenda_data_2019_1'
if folder_name != '':
    #make folders if they don't exist
    if not exists(join(RAW_EXTERNAL, folder_name)):
        os.makedirs(join(RAW_EXTERNAL, folder_name))

    if not exists(join(INTERMEDIATE, folder_name)):
        os.makedirs(join(INTERMEDIATE, folder_name))

    if not exists(join(FINAL, folder_name)):
        os.makedirs(join(FINAL, folder_name))

print('Standard variables loaded, you are good to go!')

Standard variables loaded, you are good to go!


## Here I want to develop a way to re-use the substrate to molecule conversion
The reaseon is that cirpy takes a long time to reference the datbase. I therefore want to locally store the results.

First we load up the substrates from BRENDA

In [47]:
import json
import re

filepath = join(FINAL, 'brenda_data_2019_1', 'natural_substrates_filtered.json')
with open(filepath, 'r') as f:
    filtered_subst_data = json.loads(f.read())
    

    
def remove_dl(substrate):
    '''
    Remove D-, L-, DL- and similar from the beginning of substrates
    '''
    substrate = substrate.lower()
    return re.sub('^d-|^l-|^dl-|^\(r\)-|^\(s\)-', '', substrate)
    

def get_all_ec():
    '''
    Get a list of all ec numbers
    '''
    return filtered_subst_data.keys()

    
def get_substrates(ec):
    '''
    Obtain a list of all the natural substrates for an ec number
    '''
    substrate_list = filtered_subst_data[ec]['first_substrate']
    return sorted(list(set([remove_dl(s) for s in substrate_list])))


get_substrates('1.1.3.15')
    

['2 -hydroxyisocaproate',
 '2-hydroxy-4-methylthiobutanoic acid',
 '2-hydroxybutyrate',
 '2-hydroxycaproate',
 '2-hydroxycaprylate',
 '2-hydroxyisocaproate',
 '2-hydroxyisovalerate',
 '2-hydroxyoctanoate',
 '2-hydroxypalmitate',
 '2-hydroxyvalerate',
 'alanine',
 'an (s)-2-hydroxy carboxylate',
 'glycerate',
 'glycolate',
 'glyoxylate',
 'glyoxylate thiohemiacetals',
 'homoserine',
 'isoleucine',
 'lactate',
 'leucine',
 'lysine',
 'mandelate',
 'methionine',
 'phenylalanine',
 'thiol-glyoxylate adducts',
 'tryptophan',
 'tyrosine',
 'valine']

Now we want to use cirpy to get teh molecule for each of these. BUT, first we want to check wether we have a cached file. I will be loading up the cached data and potentially adding to it. So to keep the old backups safe I want to make a new backup each day that the script is run.

In [48]:
import datetime


currentDT = datetime.datetime.now()
cached_file_dir = join(INTERMEDIATE, 'brenda_data_2019_1')
filename = 'substrate_cache.json'
date = '%s-%s-%s' % (currentDT.year, str(currentDT.month).rjust(2, '0'), str(currentDT.day).rjust(2, '0')) # YY-MM-DD


def todays_filename():
    '''
    Get the filename of todays file.
    '''
    return join(cached_file_dir, '%s_%s' % (date, filename))


def most_recent_filename():
    '''
    Return the filepath of the most recent file.
    '''
    # first make a list of all available files
    file_data = {}
    for fi in os.listdir(cached_file_dir):
        if fi.endswith(filename):
            year, month, day = fi.replace(filename, '').split('-')
            
            if file_data.get(year) is None:
                file_data[year] = {}
                
            if file_data[year].get(month) is None:
                file_data[year][month] = {}
                
            file_data[year][month][day] = join(cached_file_dir, fi)
    
    # if there is no file
    if file_data == {}:
        return None
    
    max_year = max(file_data.keys())
    max_month = max(file_data[max_year].keys())
    max_day = max(file_data[max_year][max_month].keys())
    
    return file_data[max_year][max_month][max_day]
            

def open_file(filepath):
    '''
    Open up the json file and return data structure
    '''
    with open(filepath, 'r') as f:
        data = json.loads(f.read())
    return data


def save_data(data):
    '''
    Save data to todays file.
    '''
    filepath = todays_filename()
    with open(filepath, 'w') as f:
        f.write(json.dumps(data))

    
def load_data():
    '''
    Load up existing substrate to smile data
    '''
    # if a file from today exists open it up
    filepath = todays_filename()
    if exists(filepath):
        return open_file(filepath)
    
    # if no file exists from today then make a copy of the most recent file
    else:
        old_filepath = most_recent_filename()
        
        if old_filepath is None:
            data = {}
        
        else:
            data = open_file(old_filepath) # get most recent data

        save_data(data) # save into todays filename
        return data

data = load_data()
data


{'(-)-(s)-limonene': 'CC(=C)[C@H]1CCC(=CC1)C',
 '1,2-linked glucuronic acid of non-reducing xylose-oligosaccahrides': None,
 '(3e)-3-[(4s)-4-hydroxycyclohex-2-en-1-yl]-2-oxopropanoate': None,
 '10-deoxysarpagine': 'C\\C=C/1CN2[C@H]3Cc4c([nH]c5ccccc45)[C@@H]2C[C@H]1C3CO',
 'alpha-d-xyl-(1-&gt;6)-beta-d-glc-(1-&gt;4)-[alpha-d-xyl-(1-&gt;6)]-beta-d-glc-(1-&gt;4)-[alpha-d-xyl-(1-&gt;6)]-beta-d-glc-(1-&gt;4)-d-glc': None,
 '(2-pyrrolyl)cysteine': 'OC(=O)[C@H](CS)Nc1[nH]ccc1',
 '(3-hydroxy-phenylalkanoic acid)n': None,
 '2-chloro-1-phenylethanol': 'OC(CCl)c1ccccc1',
 'neuropeptides': None,
 'ompx': None,
 'histone h3 n6-dimethyl-l-lysine4': None,
 'damaged psii d1 protein': None,
 'coronavirus polyprotein': None,
 'beta-d-galactosyl-(1-&gt;4)-l-rhamnose': None,
 '22-cholesten-3beta-ol': 'CC(C)CC=C[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CCC4C[C@@H](O)CC[C@]4(C)[C@H]3CC[C@]12C',
 'gag-pol polyprotein': None,
 'cobalt-sirohydrochlorin': None,
 'gum arabic': None,
 'n,n-dimethyltryptamine': 'CN(C)CCc1c[nH

Lets' see how many substrates we are talking about for all of BRENDA

In [49]:
all_substrates = set([])
for ec in get_all_ec():
    all_substrates.update(get_substrates(ec))
    
print('Unique substrates in BRENDA: %s' % len(all_substrates))

Unique substrates in BRENDA: 8461


Now convert these and save to file for every 100 completed ones

In [55]:
import cirpy


data = load_data()

counter = 0
for name in all_substrates:
    counter += 1
    
    if name not in data.keys():
        data[name] = cirpy.resolve(name, 'smiles')
        
    if counter % 100 == 0:
        print(counter)
        save_data(data)
save_data(data)
    

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400


In [63]:
data = load_data()

sum([s is not None for s in data.values()])

3641